<a href="https://colab.research.google.com/github/SSUKENIK/hello-world/blob/master/clean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import re  # Needed for regular expression searching


# Milestone 1 - Reading and Cleaning Data


In [ ]:
games = pd.read_html('https://en.wikipedia.org/wiki/NBA_All-Star_Game',index_col=0,parse_dates=True)[2]
pd.set_option('display.max_row',100)  # Necessary to display all the rows
games.dropna(how='any')  # Get rid of all NA rows
games           # Print out games to examine data

,Result,Host arena,Host city,Game MVP
Year,,,,
1951,"East 111, West 94",Boston Garden,"Boston, Massachusetts","Ed Macauley, Boston Celtics"
1952,"East 108, West 91",Boston Garden (2),"Boston, Massachusetts (2)","Paul Arizin, Philadelphia Warriors"
1953,"West 79, East 75",Allen County War Memorial Coliseum,"Fort Wayne, Indiana","George Mikan, Minneapolis Lakers"
1954,"East 98, West 93 (OT)",Madison Square Garden III**,"New York City, New York","Bob Cousy, Boston Celtics"
1955,"East 100, West 91",Madison Square Garden III** (2),"New York City, New York (2)","Bill Sharman, Boston Celtics"
1956,"West 108, East 94",Rochester War Memorial Coliseum,"Rochester, New York","Bob Pettit, St. Louis Hawks"
1957,"East 109, West 97",Boston Garden (3),"Boston, Massachusetts (3)","Bob Cousy (2), Boston Celtics"
1958,"East 130, West 118",St. Louis Arena,"St. Louis, Missouri","Bob Pettit (2), St. Louis Hawks"
1959,"West 124, East 108",Olympia Stadium,"Detroit, Michigan","Elgin Baylor, Minneapolis LakersBob Pettit (3)..."


# Milestone 2 - Keeping Only Data of Interest

In [ ]:
# 
#  Change year to int and use an index
#  Delete all rows that came after 2017
#
pattern = re.compile('(\d{4})')                                 # Only extract the year  2018 has an exception 
extract_date = lambda x : int(pattern.match(x).group(1))
games.index = pd.Series([extract_date(index) for index in games.index],
                        dtype=np.int16)  # Convert over the Year into integer
#
# Can use rename fuction to clean index
#
#   games = games.rename(index=extract_date)                        
games.drop(games.loc[games.index > 2017].index,inplace=True)   
games.drop(["Host arena"],axis=1,inplace=True)
del(games["Game MVP"])
#
# Change Host city to display only the city 
#
games["Host city"] = [str.strip(city[0]) for city in games["Host city"].str.split(',')]

games.tail()


,Result,Host city
2013,"West 143, East 138",Houston
2014,"East 163, West 155",New Orleans
2015,"West 163, East 158",New York City
2016,"West 196, East 173",Toronto
2017,"West 192, East 182",New Orleans


In [ ]:
#
# Filter out any years that do not contain East or West Scores
#
#  i.e. 1999	Canceled due to the league's lockout.
#
pattern = re.compile('(East|West) (\d{1,3}), (East|West) (\d{1,3})')  # Set valid score patterns
games = games.loc[games['Result'].apply(lambda x: bool(pattern.match(x))) ]


In [ ]:
#
# Build Score columns
#
def build_score_col(colname:str):
    pattern = re.compile('.*' + colname + ' (\d{1,3})')
    scores = pd.Series(games['Result'].apply(lambda x : pattern.match(x).group(1) ),dtype=np.int16)
    games.insert(2,colname,scores)
build_score_col('West')
build_score_col('East')
games = games.drop("Result",axis=1)  # Drop the results column
games.head()

,Host city,East,West
1951,Boston,111,94
1952,Boston,108,91
1953,Fort Wayne,75,79
1954,New York City,98,93
1955,New York City,100,91


# Milestone 3 - Getting counts

In [ ]:
score_diff = abs(games["East"] - games["West"])
games.insert(games.shape[1],"Diff", score_diff)
# Note - can use 
#    games['Diff'].value_counts()
#
results = pd.Series(games.groupby(['Diff'])['Diff'].count())  # Note - Series can be sorted
print(results.sort_values(ascending=False))


Diff
5     7
11    5
2     5
9     5
1     4
10    4
3     3
4     3
7     3
8     3
12    3
17    3
20    3
14    2
15    2
6     2
21    2
27    2
40    1
16    1
22    1
23    1
43    1
Name: Diff, dtype: int64


# Milestone 4 - Filtering and Grouping data

## 1. What is the maximum and minmum score difference between all games

In [ ]:
max_diff = results.index.max()
print("Maximum score difference: " + str(max_diff))
games.loc[games['Diff'] == max_diff]


Maximum score difference: 43


,Host city,East,West,Diff
1966,Cincinnati,137,94,43


In [ ]:
min_diff = results.index.min()
print("\n\nMinmum score difference: " + str(min_diff))
games.loc[games['Diff'] == min_diff]



Minmum score difference: 1


,Host city,East,West,Diff
1965,St. Louis,124,123,1
1971,San Diego,107,108,1
1977,Milwaukee,124,125,1
2001,Washington,111,110,1


## 2. If a city hosted a game more than once, what was the average score for the East and West team?

In [ ]:
group_games =games.groupby("Host city")            # Group by Host City
result = group_games[['East','West']].mean()       # Get mean for East and West columns
result['Count'] = group_games["Host city"].count() # Append Count
result.loc[result['Count'] > 1]                    # Only report on Cities that played more than 2 games



,East,West,Count
Host city,,,
Atlanta,139.000000,140.000000,2
Boston,109.750000,97.250000,4
Chicago,121.000000,108.500000,2
Denver,139.500000,130.000000,2
Houston,131.333333,135.333333,3
Inglewood,121.000000,117.500000,2
Los Angeles,130.000000,130.666667,3
New Orleans,159.666667,158.333333,3
New York City,127.000000,117.000000,5
